In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

from pyna.gc.LoopBetweenCapacitor import LoopBetweenCapacitor
from pyna.gc.motion import particle_motion_forward, particle_motion_backward, particle_motion_withoutALforce_forward, particle_motion_withALforce_forward, particle_motion_relativistic_forward
from pyna.gc.motion import gc_motion_forward
# Constants and Parameters
# q = 1.6e-19  # Charge of the particle [C]
q = 1.602176634e-19  # Accurate value of the electron charge [C]
# q = 1.602176634e-6  # Charge of the particle [C]
# m0 = 9.10938371e-31  # Accurate value of the electron mass [kg]
# m0 = 1.67e-27 * 2  # Mass of the particle [kg]
m0 = 1.672621925e-27  # Accurate value of the proton mass [kg]
# m0 = 1.672621925e-14  # Mass of the particle [kg]

EBfield = LoopBetweenCapacitor(
                    I_capacitor=lambda t: 0e-8 * np.sin(t/1e6), 
                    Q_capacitor=lambda t:-0e-2 * np.cos(t/1e6), )

# Initial conditions
# passing ion with significant center of curvature deviating from the guiding center
init_pos = [1.2, 0, 0]  # Initial position [m]
init_vel = [.0e6, 1.2e6, -.5e6]  # Initial velocity [m/s]
# init_vel = [.0e6, 1.2e7, -.5e7]  # Initial velocity [m/s]
# init_pos = [1.2, 0, 0]  # Initial position [m]
# init_vel = [1.0e6, .0e6, -.5e6]  # Initial velocity [m/s]
# init_pos = [0.8, 0, 0]  # Initial position [m]
# init_vel = [1.0e6, .0e6, -.5e6]  # Initial velocity [m/s]
init_state = init_pos + init_vel

# # Calculate initial acceleration
# c = 299792458  # speed of light in m/s
# v = np.array(init_vel)
# F_ext = q * np.array(E_and_B_at(*init_pos, t=0)[0])  # External force [N]
# gamma = 1 / np.sqrt(1 - np.linalg.norm(v)**2 / c**2)
# I = np.eye(3)
# init_acc = list( (I - np.outer(v, v) / c**2) @ F_ext / (m0 * gamma)  )
# print(init_acc)
# init_state = init_pos + init_vel + init_acc

print('E and B at (0, 0, 0):', EBfield.E_and_B_at(*init_pos, t=0) )
B0 = np.linalg.norm(EBfield.B_at(*init_pos, t=0))  # Central magnetic field [T]
omega_c0 = q * B0 / m0


# Event function to stop the solver when abs(z) > d_capacitor / 2
def cb(t, state, p):
    # x, y, z, _, _, _ = state
    # x, y, z, vx, vy, vz, _, _, _ = state
    return abs(state[2]) - EBfield.d_capacitor / 2

cb.terminal = True
cb.direction = 0

print("Central magnetic field [T]: ", B0)
print("Cyclotron angular frequency [rad/s]: ", omega_c0)
print("Cyclotron period [s]: ", 2 * np.pi / omega_c0)

t_span = (0, 2 * np.pi / omega_c0 * 100)  # Time span [s]
# Solve the equations of motion
Xtrj_forward_sol = solve_ivp(particle_motion_forward, t_span, init_state, dense_output=True, 
                     method='RK45', max_step= 2*np.pi / (omega_c0) / 1080, events=cb,
                    args=[[q, m0, EBfield],])

v_parallel = np.dot(init_vel, EBfield.hatb_at(*init_pos, t=0.0) )
init_rho = m0/q * np.cross(EBfield.B_at(*init_pos, t=0), init_vel - EBfield.vE_at(*init_pos, t=0)) / EBfield.B_abs_at(*init_pos, t=0)**2
init_state = list(init_pos - init_rho)  + [v_parallel,]
mu = m0 * (np.dot(init_vel, init_vel) - v_parallel**2) / (2 * EBfield.B_abs_at(*init_pos, t=0.0) )
GCtrj_forward_sol = solve_ivp(gc_motion_forward, t_span, init_state, dense_output=True, 
                     method='RK45', max_step= 2*np.pi / (omega_c0) / 3, events=cb,
                    args=[[q, m0, mu, EBfield],])
# Xtrj_backward_sol = solve_ivp(particle_motion_backward, t_span, init_state, dense_output=True, 
#                      method='RK45', max_step= 2*np.pi / (omega_c0) / 360, events=cb,
#                     args=[[q, m0, EBfield],])
# Xtrj_forward_sol = solve_ivp(particle_motion_withoutALforce_forward, t_span, init_state, dense_output=True, 
#                      method='RK45', max_step= 2*np.pi / (omega_c0) / 360, events=cb,
#                     args=[[q, m0, EBfield],])

# Xtrj_forward_sol = solve_ivp(particle_motion_withALforce_forward, t_span, init_state, dense_output=True, 
#                      method='RK45', max_step= 2*np.pi / (omega_c0) / 360, events=cb,
#                     args=[[q, m0, EBfield],])
# Xtrj_forward_sol = solve_ivp(particle_motion_relativistic_forward, t_span, init_state, dense_output=True, 
#                      method='RK45', max_step= 2*np.pi / (omega_c0) / 3600, events=cb,
#                     args=[[q, m0, EBfield],])


In [ ]:

# Extract the trajectory
x, y, z = Xtrj_forward_sol.y[0], Xtrj_forward_sol.y[1], Xtrj_forward_sol.y[2]
t = Xtrj_forward_sol.t
# x, y, z = GCtrj_forward_sol.y[0], GCtrj_forward_sol.y[1], GCtrj_forward_sol.y[2]
# t = GCtrj_forward_sol.t


In [ ]:
# particle_motion_relativistic_forward(0, init_state, [q, m0, E_and_B_at])

In [ ]:
from scipy.integrate import cumulative_trapezoid
from scipy.interpolate import interp1d

# Compute the speed at each time step using central difference
speed = np.sqrt((x[2:] - x[:-2])**2 + (y[2:] - y[:-2])**2 + (z[2:] - z[:-2])**2) / (t[2:] - t[:-2])

# Compute the arc length s(t) by integrating the speed
s = cumulative_trapezoid(speed, t[1:-1], initial=0)

# Interpolate to find t(s)
t_of_s = interp1d(s, t[1:-1], kind='linear', fill_value="extrapolate")

# Parameterize the trajectory by arc length s
x_of_s = interp1d(s, x[1:-1], kind='linear', fill_value="extrapolate")
y_of_s = interp1d(s, y[1:-1], kind='linear', fill_value="extrapolate")
z_of_s = interp1d(s, z[1:-1], kind='linear', fill_value="extrapolate")

# Compute the derivatives with respect to s
dx_ds = np.gradient(x_of_s(s), s)
dy_ds = np.gradient(y_of_s(s), s)
dz_ds = np.gradient(z_of_s(s), s)

d2x_ds2 = np.gradient(dx_ds, s)
d2y_ds2 = np.gradient(dy_ds, s)
d2z_ds2 = np.gradient(dz_ds, s)
kappa = np.vstack((d2x_ds2, d2y_ds2, d2z_ds2)).T
hatb_onXtrj = np.vstack([EBfield.hatb_at(x_of_s(_s), y_of_s(_s), z_of_s(_s), t=t_of_s(_s)) for _s in s])

kappa_ll = kappa - np.sum(kappa * hatb_onXtrj, axis=1)[:, np.newaxis] * hatb_onXtrj
kappa_ll_abs = np.sqrt(np.sum(kappa_ll**2, axis=1))

# Compute the curvature κ(s)
kappa_abs = np.sqrt(d2x_ds2**2 + d2y_ds2**2 + d2z_ds2**2)

# Compute the center of curvature C(s)
T_prime = np.vstack((d2x_ds2, d2y_ds2, d2z_ds2)).T
C_of_s = np.vstack((x_of_s(s), y_of_s(s), z_of_s(s))).T + (1 / kappa_abs**2)[:, np.newaxis] * T_prime
C_perp_of_s = np.vstack((x_of_s(s), y_of_s(s), z_of_s(s))).T + (1 / kappa_abs**2)[:, np.newaxis] * kappa_ll

# Define lambda functions for curvature and center of curvature
curvature = lambda s: interp1d(t[1:-1], kappa_abs, kind='linear', fill_value="extrapolate")(s)
center_of_curvature = lambda s: np.vstack((
    interp1d(t[1:-1], C_of_s[:, 0], kind='linear', fill_value="extrapolate")(s),
    interp1d(t[1:-1], C_of_s[:, 1], kind='linear', fill_value="extrapolate")(s),
    interp1d(t[1:-1], C_of_s[:, 2], kind='linear', fill_value="extrapolate")(s)
)).T

# Example usage
# s_values = np.linspace(s[0], s[-1], num=1000)
curvatures = curvature(t[2])
centers_of_curvature = center_of_curvature(t[2])

print("Curvatures:", curvatures)
print("Centers of Curvature:", centers_of_curvature)

v_parallel = np.array([np.dot(EBfield.E_and_B_at(x[i+1], y[i+1], z[i+1], t=t[i+1])[1], [dx_ds[i], dy_ds[i], dz_ds[i]]) for i in range(len(t[1:-1]))])


In [ ]:
np.sum(kappa * hatb_onXtrj, axis=1).shape

In [ ]:

# Plot the trajectory
plt.figure(figsize=(10, 6))
firstN = len(t) // 1
# plt.plot(t[:firstN], x[:firstN] - x[0], label='x')
# plt.plot(t[:firstN], y[:firstN] - y[0], label='y')
plt.plot(t[:firstN], z[:firstN] - z[0], label='z')
firstN = len(GCtrj_forward_sol.t) // 1
# plt.plot(GCtrj_forward_sol.t[:firstN], GCtrj_forward_sol.y[0,:firstN] - GCtrj_forward_sol.y[0,0], label='GC x')
# plt.plot(GCtrj_forward_sol.t[:firstN], GCtrj_forward_sol.y[1,:firstN] - GCtrj_forward_sol.y[1,0], label='GC y')
plt.plot(GCtrj_forward_sol.t[:firstN], GCtrj_forward_sol.y[2,:firstN] - GCtrj_forward_sol.y[2,0], label='GC z')
plt.xlabel('t [s]')
plt.ylabel('y [m]')
plt.title('Guiding Center Theory Verification')
plt.legend()
plt.grid()
plt.show()

In [ ]:
import plotly.graph_objects as go
# import plotly.express as px


# Create a 3D scatter plot in line mode
fig = go.Figure(data=[
    go.Scatter3d(
        x=EBfield.radius_coil * np.cos(np.linspace(.0, 2*np.pi, num=500, endpoint=True,)), 
        y=EBfield.radius_coil * np.sin(np.linspace(.0, 2*np.pi, num=500, endpoint=True,)), 
        z=np.zeros(500),
        mode='lines',
        line=dict(color="rgb(166,86,40)", width=10),
        name='Coil',
    ),
    go.Scatter3d(
        x=Xtrj_forward_sol.y[0][1:-1], y=Xtrj_forward_sol.y[1][1:-1], z=Xtrj_forward_sol.y[2][1:-1],
        mode='lines',
        line=dict(
            color=v_parallel,
            colorscale='RdBu_r',
            cmin=-np.max(np.abs(v_parallel)),
            cmax= np.max(np.abs(v_parallel)),
            width=3,
        ),
        name='Particle Trajectory w/ AL force',
    ),
    # go.Scatter3d(
    #     x=C_of_s[:,0], y=C_of_s[:,1], z=C_of_s[:,2],
    #     mode='lines',
    #     line=dict(color='black', width=1),
    #     name='Center of Curvature',
    # ),
    go.Scatter3d(
        x=C_perp_of_s[:,0], y=C_perp_of_s[:,1], z=C_perp_of_s[:,2],
        mode='lines',
        line=dict(color='black', width=1),
        name='Center of Curvature',
    ),
    # go.Scatter3d(
    #     x=GCtrj_forward_sol.y[0][:], y=GCtrj_forward_sol.y[1][:], z=GCtrj_forward_sol.y[2][:],
    #     mode='lines',
    #     line=dict(
    #         color=GCtrj_forward_sol.y[3][:],
    #         colorscale='RdBu_r',
    #         cmin=-np.max(np.abs(GCtrj_forward_sol.y[3][:])),
    #         cmax= np.max(np.abs(GCtrj_forward_sol.y[3][:])),
    #         width=3,
    #     ),
    #     name='Particle Trajectory w/ AL force',
    # ),
    # go.Scatter3d(
    #     x=Xtrj_backward_sol.y[0][1:-1], y=Xtrj_backward_sol.y[1][1:-1], z=Xtrj_backward_sol.y[2][1:-1],
    #     mode='lines',
    #     line=dict(color='black', width=3),
    #     name='Particle Trajectory',
    # ),
])


# Set plot title and axis labels
fig.update_layout(
    title='Particle Motion Trajectory',
    scene=dict(
        xaxis_title='x [m]',
        yaxis_title='y [m]',
        zaxis_title='z [m]',
        aspectmode='data',
        xaxis = dict(
            backgroundcolor="rgba(0, 0, 0, 0.02)",
            gridcolor="lightgrey",
            showbackground=False,
            zerolinecolor="black",),
        yaxis = dict(
            backgroundcolor="rgba(0, 0, 0, 0.0)",
            gridcolor="lightgrey",
            showbackground=False,
            zerolinecolor="black"),
        zaxis = dict(
            backgroundcolor="rgba(0, 0, 0, 0.0)",
            gridcolor="lightgrey",
            showbackground=False,
            zerolinecolor="black",),
    ),
    width=2000,
    height=1200
)
# fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
# Show the plot
fig.show()

In [ ]:
# fig.add_trace(
#     go.Scatter3d(
#         x=Xtrj_forward_sol.y[0][1:-1], y=Xtrj_forward_sol.y[1][1:-1], z=Xtrj_forward_sol.y[2][1:-1],
#         mode='lines',
#         line=dict(
#             color=v_parallel,
#             colorscale='RdBu_r',
#             cmin=-np.max(np.abs(v_parallel)),
#             cmax= np.max(np.abs(v_parallel)),
#             width=3,
#         ),
#         name='Particle Trajectory non-relativistic',
#     ),
# )